In [ ]:
##import libraries
import numpy as np
import os
import pandas as pd
import scipy as sp
#pip install eeglabio ##if you can not run eeglab or have not use it before, delete the hashtag
import eeglabio
import mne
#pip install emd ##if you can not run the code below as it can not find the functions/you have not run this library before, please delete the hashtag
from emd.sift import mask_sift
from emd.spectra import define_hist_bins, frequency_transform
import matplotlib.pyplot as plt
from scipy.fft import fft, ifft
from scipy.signal import welch
from scipy.io import savemat
import json
from sklearn.decomposition import FastICA
import joblib
from mne.preprocessing import ICA
import mne, time
from datetime import datetime
from mne.io import read_raw_edf
from Basic_function import group_consecutives, get_start_timestamp, chunks, ica_pipe, remove_imfs, mask_sift_emd,plot_beforeafter,plot_psd,plot_fft_beforeafter
from EEG import EEG

In [ ]:
##redirect to the folder contain the edf files
path = r'D:\data video\CutControl'

In [ ]:
##IMPORT DATA ##please label data in a system for easy input
subject_id= 'C01'
path_edf = os.path.join(path, f'{subject_id}.edf') ##call out the file according to its name - subject id
eeg = read_raw_edf(path_edf,preload=True,exclude=('Status')) ##this is compulsory to read the edf files ##exclude 'status' as it is stimulus channel created due to import of event.csv in eeglab(MATLAB)
raw_data=eeg.get_data() 

In [ ]:
## CHECKPOINT-you can get the metadata included in the file and a list of all channels
#info = eeg.info
#channels = eeg.ch_names
#print(channels)
#print(np.shape(raw_data))

In [ ]:
##Masked EMD for all channels, max imfs = 12 means that each channel signal was decomposed to 12 IMF 
imfs_all = mask_sift_emd(data = raw_data,
                duration = None,
                channel = None,
                sample_rate = 200,
                mask_freqs = 50,
                ret_mask_freq = True,
                max_imfs = 12)
##RECONSTRUCT THE DATA AFTER ELIMINATE THE IMFS COMPONENT
eeg_after_emd = eeg.copy()
eeg_after_emd.apply_function(remove_imfs, imfs = imfs_all, idx = [0, 9, 10, 11], channel_wise=False)
data_after_emd = eeg_after_emd.get_data().T

In [ ]:
## CHECKPOINT-use the following code to check the arrangement of your datasets. To plot beforeandafter, they need to be in same shape
#np.shape(raw_data)
#np.shape(data_after_emd)

In [ ]:
## CHECKPOINT-plot it out to check effect of EMD on your data. Keep the duration short as too long can cause the code lagging.
plot_beforeafter(data1=raw_data[:,0:30*200].T,
                   data2=data_after_emd[0:30*200,:],
                   eeg=eeg,
                   title='Before&after',
                     figsize=(64,48),
                     fs=200,
                  y_unit='uV',
                  savefig = os.path.join(path,f'{subject_id}.jpg')) ##here we only plot out 30-secs of data

In [ ]:
##EXPORT the post-EMD as .mat file ##file will be automatically saved in the home folder of your JUPYTER Notebook. Get out and dowload it
from scipy import io
io.savemat('export.mat', {"data": data_after_emd})